In [1]:
# !pip install transformers

In [1]:
import re
import gc
import os
import pickle
import torch
import random
from torch.nn import BCEWithLogitsLoss, BCELoss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold,StratifiedKFold, train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm, trange
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, roc_curve
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report,f1_score
from transformers import AutoModel,BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup,BertForSequenceClassification
from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold,KFold
from torch.nn.functional import softmax
from transformers import logging
logging.set_verbosity_warning()
from transformers import AutoConfig, AutoTokenizer, AutoModel
import time
import datetime
import seaborn as sns
from preprocess_ import preprocess

# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_colwidth',-1)
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

In [2]:
def cleaner(x):
    x = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", x).strip()
    x = re.sub('([.,!?؟()!\\/\+\-\=\|:"،])', r' \1 ', x)

    x = x.replace('\u200c', ' ').replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
    
    x = re.sub('\/', ' ', x)
    x = re.sub('«', ' ', x)
    x = re.sub('»', ' ', x)
    
    x = re.sub('\s{2,}', ' ', x)
    return x.strip()

In [3]:
files = os.listdir(r"E:\Work\Sigmoyd\author identification\zoomit\batch data")
li = []


for i in files:
    df = pd.read_csv(f"E:\\Work\\Sigmoyd\\author identification\\zoomit\\batch data\\{i}", index_col=0)
    li.append(df)
    
    
df = pd.concat(li)

best_authors = df.author.value_counts().head(23).index
df = df[df.author.isin(best_authors)]
df = df.reset_index(drop=True)

# df['text'] = df['title'] + ' . ' + df['lead'] + ' . ' + df['text']

# df['text'] = df['title']

prep = preprocess()
df['text'] = df['text'].astype(str).progress_apply(lambda x: prep.clean(x))

100%|███████████████████████████████████████████████████████████████████████████| 43458/43458 [01:46<00:00, 407.75it/s]


In [4]:
cat_mapper = dict(zip(df.author.unique() , range(23)))

df.author = df.author.map(cat_mapper)

In [5]:
device = torch.device("cuda")
n_gpu = torch.cuda.device_count()

In [6]:
f1 = df.copy()

In [7]:
f1=f1[~f1['author'].isna()]

In [8]:
f1=f1[~f1['text'].isna()]

In [9]:
X = f1.loc[:,["text"]]
Y = f1.author
# test = df_test

In [10]:
Y.value_counts()

3     7570
20    4066
2     2351
21    2269
9     2264
15    2207
1     2088
11    1991
17    1776
18    1706
5     1373
10    1317
16    1257
14    1202
6     1164
4     1158
22    1152
0     1151
8     1143
12    1136
19    1073
7     1028
13    1016
Name: author, dtype: int64

In [13]:
config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
f1.author.nunique()

23

In [15]:
seq_len = [len(i.split()) for i in X['text']]
# pd.Series(seq_len).hist(bins = 300)

In [16]:
max_length = 512 #512 #32 #128 
num_labels = 23

In [17]:
X['len'] = seq_len
X.loc[X['len']>max_length,'text'] = X.loc[X['len']>max_length,'text'].str[-max_length:]
X = X['text']


In [18]:
def make_model():
    model = BertForSequenceClassification.from_pretrained("HooshvareLab/bert-base-parsbert-uncased",num_labels=23,output_hidden_states = False)
    model = model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,eps = 1e-8,correct_bias=True)
    return model, optimizer

In [19]:
def flat_evaluation(preds, preds_, labels):
    preds = np.array(preds)
    labels = np.array(labels)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = np.sum(pred_flat == labels_flat) / len(labels_flat)
    f1_micro = f1_score(labels_flat,pred_flat,average="micro")
    f1_macro = f1_score(labels_flat,pred_flat,average="macro")

    preds_ = [i[1] for i in preds_]
#     auc = roc_auc_score(labels_flat,preds_)

    return acc, f1_micro ,f1_macro #,auc

In [20]:


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [21]:
all_LOGITS = []
all_LOGITS_ = []
all_LABELS = []



batch_size = 8
train_loss_set = []
loss_values = []
ACC,AUC =[],[]
pred=[]
valid=[]
predprob_li_logits=[]
predprob_li_logits_=[]
predprob_li_labels=[]
counter = -1
epochs = 4
# skf=KFold(n_splits=5,random_state=2)
kf = StratifiedKFold(n_splits=5,random_state=2)

fold=0
for i,(train_index, test_index) in enumerate(kf.split(X , Y)):
#     counter+=1
#     if counter==0:
#         continue
    model, optimizer = make_model()
    X_train, X_test = X.iloc[train_index].values, X.iloc[test_index].values
    y_train, y_test = Y.iloc[train_index].values, Y.iloc[test_index].values
    valid.append(test_index)
    encodings_train = tokenizer.batch_encode_plus(X_train.tolist(),max_length=max_length,pad_to_max_length=True)
    encodings_test = tokenizer.batch_encode_plus(X_test.tolist(),max_length=max_length,pad_to_max_length=True)
#     print("1")                                           
    input_ids = encodings_train['input_ids']# tokenized and encoded sentences
    token_type_ids = encodings_train['token_type_ids'] # token type ids
    attention_masks = encodings_train['attention_mask'] # attention masks
#     print("2") 
    input_ids_ = encodings_test['input_ids']# tokenized and encoded sentences
    token_type_ids_ = encodings_test['token_type_ids'] # token type ids
    attention_masks_ = encodings_test['attention_mask']
#     print("3") 
    train_inputs = torch.tensor(input_ids)
    train_labels = torch.tensor(y_train)
    train_masks = torch.tensor(attention_masks)
    train_token_types = torch.tensor(token_type_ids)
#     print("4") 
    test_inputs = torch.tensor(input_ids_)
    test_labels = torch.tensor(y_test)
    test_masks = torch.tensor(attention_masks_)
    test_token_types = torch.tensor(token_type_ids_)
#     print("5")

    train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
#     print("6")
    validation_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
#     print("7")
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
#     print("8")
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_loss = 0
        model.train()
        

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 400 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()        
            outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

            loss = outputs[0]
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

        print("")
        print("  Average training loss: {0:.5f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()
        model.eval()

        # Tracking variables 
        eval_loss, eval_accuracy,eval_f1,eval_auc = 0, 0,0,0
        nb_eval_steps, nb_f1_eval_steps ,nb_eval_examples, nb_auc_eval_steps = 0, 0 ,0, 0

        LOGITS, LOGITS_, LABELS = [], [], []
        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels ,b_token_types= batch
            with torch.no_grad():        
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)

            logits = outputs[0]
            logits_ = softmax(logits)
#             print(logits)
            # Move logits and labels to CPU
            LOGITS += list(logits.detach().cpu().numpy().tolist())
            LOGITS_ += list(logits_.detach().cpu().numpy())
            LABELS += list(b_labels.to('cpu').numpy().tolist())
        predprob_li_logits.append(LOGITS)
        predprob_li_logits_.append(LOGITS_)
        predprob_li_labels.append(LABELS)
          
        eval_accuracy, eval_f1, eval_auc = flat_evaluation(LOGITS, LOGITS_, LABELS)
        print("  Accuracy: {0:.5f}".format(eval_accuracy))
        print("  f1: {0:.5f}".format(eval_f1))
        print("  AUC: {0:.5f}".format(eval_auc))
        print("  Validation took: {:}".format(format_time(time.time() - t0)))
        torch.save(model.state_dict(),'./model_{}_{}'.format(i,epoch_i))
    all_LOGITS += LOGITS
    all_LOGITS_ += LOGITS_
    all_LABELS += LABELS
    print("")
    print("Training complete!")    
    
    del X_train, y_train
    gc.collect()

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica


======== Epoch 1 / 4 ========
Training...
  Batch   400  of  4,345.    Elapsed: 0:02:38.
  Batch   800  of  4,345.    Elapsed: 0:05:13.


KeyboardInterrupt: 

In [ ]:
#   Batch   800  of  4,345.    Elapsed: 0:05:09.


In [23]:
(79768 + 87868)/2

83818.0

In [ ]:
def flat_evaluation_(preds, preds_, labels):
    preds = np.array(preds)
    labels = np.array(labels)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = np.sum(pred_flat == labels_flat) / len(labels_flat)
    f1_micro = f1_score(labels_flat,pred_flat,average="micro")
    f1_macro = f1_score(labels_flat,pred_flat,average="macro")

    preds_ = [i[1] for i in preds_]
    return pred_flat

In [ ]:
a = flat_evaluation_(all_LOGITS, all_LOGITS_, all_LABELS)

plt.figure(figsize=(16,12))
sns.heatmap(confusion_matrix(all_LABELS , a) , annot=True, fmt='g' ,
           xticklabels=cat_mapper,
           yticklabels=cat_mapper , cmap="rocket")

In [ ]:
flat_evaluation(LOGITS, LOGITS_, LABELS)

In [ ]:
LOGITS_[0]

In [ ]:
LABELS[0]

In [ ]:
predproba_list=[]
for i in predprob_li_logits_:
    for j in i:
        predproba_list.append(j[0])
        

In [ ]:
valid_index=[]
for i in valid:
    for j in i:
        valid_index.append(j)

In [ ]:
a=pd.DataFrame()
a['index']=valid_index
a['pred_just_text']=predproba_list

In [ ]:
b=pd.merge(a.iloc[:,[1]],f1.reset_index(drop=True),right_index=True , left_index=True)
b=b[['token','pred_just_text']]

In [ ]:
t=test['text'].values

In [ ]:

encodings_test = tokenizer.batch_encode_plus(t.tolist(),max_length=max_length,pad_to_max_length=True)
                                                 

input_ids_ = encodings_test['input_ids']# tokenized and encoded sentences
token_type_ids_ = encodings_test['token_type_ids'] # token type ids
attention_masks_ = encodings_test['attention_mask']


test_inputs = torch.tensor(input_ids_)

test_masks = torch.tensor(attention_masks_)
test_token_types = torch.tensor(token_type_ids_)


validation_data = TensorDataset(test_inputs, test_masks, test_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
model.load_state_dict(torch.load('model_0_0'))

In [ ]:

LOGITS_=[]
for batch in validation_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask ,b_token_types= batch
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits_ = softmax(logits)
#             print(logits)
    # Move logits and labels to CPU

    LOGITS_ += list(logits_.detach().cpu().numpy().tolist())
        
     
    
        

In [ ]:
model.load_state_dict(torch.load('model_1_0'))

In [ ]:

LOGITS_1=[]
for batch in validation_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask ,b_token_types= batch
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits_ = softmax(logits)
#             print(logits)
    # Move logits and labels to CPU

    LOGITS_1 += list(logits_.detach().cpu().numpy().tolist())
        
     
    
        

In [ ]:
model.load_state_dict(torch.load('model_2_0'))

In [ ]:

LOGITS_2=[]
for batch in validation_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask ,b_token_types= batch
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits_ = softmax(logits)
#             print(logits)
    # Move logits and labels to CPU

    LOGITS_2 += list(logits_.detach().cpu().numpy().tolist())
        
     
    
        

In [ ]:
model.load_state_dict(torch.load('model_3_0'))

In [ ]:

LOGITS_3=[]
for batch in validation_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask ,b_token_types= batch
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits_ = softmax(logits)
#             print(logits)
    # Move logits and labels to CPU

    LOGITS_3 += list(logits_.detach().cpu().numpy().tolist())
        
     
    
        

In [ ]:
model.load_state_dict(torch.load('model_4_0'))

In [ ]:

LOGITS_4=[]
for batch in validation_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask ,b_token_types= batch
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits_ = softmax(logits)
#             print(logits)
    # Move logits and labels to CPU

    LOGITS_4 += list(logits_.detach().cpu().numpy().tolist())
        
     
    
        

In [ ]:
bert_predictions=np.mean((LOGITS_,LOGITS_1,LOGITS_2,LOGITS_3,LOGITS_4),axis=0)

In [ ]:
logits_=[]
for i in bert_predictions:
    logits_.append(i[0])

In [ ]:
sub = pd.DataFrame({'token':test['token'].values ,'prediction':logits_})
sub.head()

In [ ]:
b.to_csv('train_just_bert.csv',index=False)
sub.to_csv('submission_just_bert.csv', index=False)